# Geoprocessing

Create a dataset that expresses a narrative from part of your daily life, either now or in the past. This can be based on a mental map of your experience in New York / another city, based on geolocated map tracks (i.e. Google Maps history), or some other source. Based on the data type (point, line, polygon), consider how it would or could relate to other datasets that lend themselves to describing your mental image of the city- subway routes, the location of open space, your favorite coffee cart, etc. We will spend time in the next class relating these kinds of datasets together. 
- Use https://geojson.io/, https://play.placemark.io/, QGIS, or another software  to create your dataset
- **You must submit a GeoJSON file of your dataset, along with a proposed related dataset, via [tk], along with a proposed workflow for relating the two (expressed as a diagram)**. Ideally the related dataset will be something you have access to, but if not, describe how you would propose creating it. Come prepared to discuss, we will talk through a couple of examples next class.

Data Collection

Datasets:
Old Craft Communities (GeoJSON/CSV)
Train Stops (GeoJSON)
Data Cleaning and Preparation
--> 
Load and inspect datasets.
Ensure consistency in coordinate systems.
Spatial Analysis
-->
Calculate the distance between each craft community and the nearest train stop.
Identify craft communities within a certain radius of train stops.
Visualization
-->
Create an interactive map showing craft communities and train stops.
Use color-coding or markers to indicate proximity to train stops.
Narrative and Insights
-->
Discuss the accessibility of craft communities via public transport.
Highlight areas with high accessibility and potential gaps.


Accessibility Challenges for Old Craft Communities in Bangkok
Narrative
This story aims to highlight the accessibility issues faced by the old craft communities in Bangkok due to the misalignment with public transportation routes, particularly train stops. By analyzing the proximity of these communities to train stops, we can understand the commuting difficulties and propose potential improvements.


In [41]:
import geopandas as gpd

# Load GeoJSON files
craft_communities_gdf = gpd.read_file('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/old craft communities.geojson')
train_stops_gdf = gpd.read_file('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/train_stops.geojson')

# Inspect data
print(craft_communities_gdf.head())
print(train_stops_gdf.head())


   ID         Name                                        Description  \
0   1   Talad Noi   The prominence in craftsmanship and skilled wo...   
1   2     Pahurat   The historic commercial district of Pahurat in...   
2   3   Kudee Jeen  This community is characterized by its harmoni...   
3   4  Narng Lerng  Ban Nang Loeng, also known as Ban E-Leng, deri...   
4   5       Ban Bu  This community, with a history of over 200 yea...   

    Country Province                    geometry  
0  Thailand  Bangkok  POINT (100.51311 13.73333)  
1  Thailand  Bangkok  POINT (100.49953 13.74454)  
2  Thailand  Bangkok  POINT (100.47936 13.73173)  
3  Thailand  Bangkok  POINT (100.51214 13.75926)  
4  Thailand  Bangkok  POINT (100.47893 13.76234)  
   ID  Name Description Country Province id_station  id_train  \
0 NaN  None        None    None     None       None       1.0   
1 NaN  None        None    None     None       None       2.0   
2 NaN  None        None    None     None       None       3.

In [42]:
# Ensure both GeoDataFrames use the same coordinate system
craft_communities_gdf = craft_communities_gdf.to_crs(epsg=4326)
train_stops_gdf = train_stops_gdf.to_crs(epsg=4326)


In [43]:
from shapely.geometry import Point
from geopy.distance import geodesic

def calculate_distance(row, stops_gdf):
    community_point = (row.geometry.y, row.geometry.x)
    distances = stops_gdf.geometry.apply(lambda x: geodesic(community_point, (x.y, x.x)).meters)
    return distances.min()

craft_communities_gdf['nearest_stop_distance'] = craft_communities_gdf.apply(calculate_distance, axis=1, stops_gdf=train_stops_gdf)


In [44]:
import folium
from folium.plugins import HeatMap

# Initialize map
map_center = [13.7563, 100.5018]
map_bangkok = folium.Map(location=map_center, zoom_start=12)

# Add train stops
for _, row in train_stops_gdf.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], icon=folium.Icon(color='blue')).add_to(map_bangkok)

# Add craft communities with proximity information
for _, row in craft_communities_gdf.iterrows():
    color = 'green' if row['nearest_stop_distance'] < 1000 else 'red'
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup=f"Distance to nearest stop: {row['nearest_stop_distance']:.2f} meters", icon=folium.Icon(color=color)).add_to(map_bangkok)

# Save map
map_bangkok.save('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/Craft_Communities_Train_Accessibility_Map.html')


In [45]:
import folium
from folium.plugins import HeatMap

# Initialize map
map_center = [13.7563, 100.5018]
map_bangkok = folium.Map(location=map_center, zoom_start=12)

# Add train stops
for _, row in train_stops_gdf.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], icon=folium.Icon(color='blue')).add_to(map_bangkok)

# Add craft communities with proximity information
for _, row in craft_communities_gdf.iterrows():
    color = 'green' if row['nearest_stop_distance'] < 1000 else 'red'
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup=f"Distance to nearest stop: {row['nearest_stop_distance']:.2f} meters", icon=folium.Icon(color=color)).add_to(map_bangkok)

# Display the map
map_bangkok


In [46]:
import geopandas as gpd
import pandas as pd

# Load GeoJSON files
craft_communities_gdf = gpd.read_file('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/old craft communities.geojson')
train_stops_gdf = gpd.read_file('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/train_stops.geojson')

# Inspect data
print(craft_communities_gdf.head())
print(train_stops_gdf.head())


   ID         Name                                        Description  \
0   1   Talad Noi   The prominence in craftsmanship and skilled wo...   
1   2     Pahurat   The historic commercial district of Pahurat in...   
2   3   Kudee Jeen  This community is characterized by its harmoni...   
3   4  Narng Lerng  Ban Nang Loeng, also known as Ban E-Leng, deri...   
4   5       Ban Bu  This community, with a history of over 200 yea...   

    Country Province                    geometry  
0  Thailand  Bangkok  POINT (100.51311 13.73333)  
1  Thailand  Bangkok  POINT (100.49953 13.74454)  
2  Thailand  Bangkok  POINT (100.47936 13.73173)  
3  Thailand  Bangkok  POINT (100.51214 13.75926)  
4  Thailand  Bangkok  POINT (100.47893 13.76234)  
   ID  Name Description Country Province id_station  id_train  \
0 NaN  None        None    None     None       None       1.0   
1 NaN  None        None    None     None       None       2.0   
2 NaN  None        None    None     None       None       3.

In [47]:
# Ensure both GeoDataFrames use the same coordinate system
craft_communities_gdf = craft_communities_gdf.to_crs(epsg=4326)
train_stops_gdf = train_stops_gdf.to_crs(epsg=4326)


In [48]:
from shapely.geometry import Point
from geopy.distance import geodesic

def calculate_distance(row, stops_gdf):
    community_point = (row.geometry.y, row.geometry.x)
    distances = stops_gdf.geometry.apply(lambda x: geodesic(community_point, (x.y, x.x)).meters)
    return distances.min()

craft_communities_gdf['nearest_stop_distance'] = craft_communities_gdf.apply(calculate_distance, axis=1, stops_gdf=train_stops_gdf)


In [49]:
import folium

# Initialize map
map_center = [13.7563, 100.5018]
map_bangkok = folium.Map(location=map_center, zoom_start=12)

# Add train stop lines
train_lines = folium.FeatureGroup(name="Train Lines")
for _, row in train_stops_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(train_lines)
train_lines.add_to(map_bangkok)

# Add craft communities with proximity information
communities = folium.FeatureGroup(name="Craft Communities")
for _, row in craft_communities_gdf.iterrows():
    color = 'green' if row['nearest_stop_distance'] < 1000 else 'red'
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"Distance to nearest stop: {row['nearest_stop_distance']:.2f} meters"
    ).add_to(communities)
communities.add_to(map_bangkok)

# Add layer control
folium.LayerControl().add_to(map_bangkok)

# Save map
map_bangkok.save('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/Craft_Communities_Train_Accessibility_Map.html')


In [50]:
# Display the map
map_bangkok

In [51]:
import geopandas as gpd
import folium
from geopy.distance import geodesic
from shapely.geometry import Point

# Load GeoJSON files
craft_communities_gdf = gpd.read_file('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/old craft communities.geojson')
train_stops_gdf = gpd.read_file('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/train_stops.geojson')

# Ensure both GeoDataFrames use the same coordinate system
craft_communities_gdf = craft_communities_gdf.to_crs(epsg=4326)
train_stops_gdf = train_stops_gdf.to_crs(epsg=4326)

# Calculate the distance from each craft community to the nearest train stop
def calculate_distance(row, stops_gdf):
    community_point = (row.geometry.y, row.geometry.x)
    distances = stops_gdf.geometry.apply(lambda x: geodesic(community_point, (x.y, x.x)).meters)
    return distances.min()

craft_communities_gdf['nearest_stop_distance'] = craft_communities_gdf.apply(calculate_distance, axis=1, stops_gdf=train_stops_gdf)

# Create an interactive map with train stops as lines and communities as dots
map_center = [13.7563, 100.5018]
map_bangkok = folium.Map(location=map_center, zoom_start=12)

# Add train stop lines
train_lines = folium.FeatureGroup(name="Train Lines")
for _, row in train_stops_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(train_lines)
train_lines.add_to(map_bangkok)

# Add craft communities with proximity information
communities = folium.FeatureGroup(name="Craft Communities")
for _, row in craft_communities_gdf.iterrows():
    color = 'green' if row['nearest_stop_distance'] < 1000 else 'red'
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"Distance to nearest stop: {row['nearest_stop_distance']:.2f} meters"
    ).add_to(communities)
communities.add_to(map_bangkok)

# Add layer control
folium.LayerControl().add_to(map_bangkok)

# Save map
map_bangkok.save('/Users/panyanusjansomwong/Desktop/cdp_summer2024/cdp-mapping-systems/Data/Craft_Communities_Train_Accessibility_Map.html')


In [52]:
# Display the map
map_bangkok